In [1]:
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib import gridspec
import matplotlib.animation as animation
from matplotlib import style
import tkinter as tk
from tkinter import ttk
from functools import partial
import numpy as np

In [2]:
class channel:
    """The channel class represents a MIPS output voltage which
    applies the voltage on the SLIM instrument."""
    
    def __init__(self, description, voltage):
        self.description = description
        self.voltage = voltage
    
    def set_voltage(self, new_voltage):
        self.voltage = new_voltage

class region:
    """The region class represents the space between two voltage inputs. 
    Over a defined length, the two voltage inputs create the separation field."""
    def __init__(self, description, length, field_strength, rungs=None):
        self.description = description
        self.length = length
        self.field_strength = field_strength
        self.rungs=rungs
        
    def set_length(self, new_length):
        self.length = new_length
        
    def set_field_strength(self, new_field_strength):
        self.field_strength = new_field_strength
        
    def voltage_drop(self):
        voltage_drop = self.field_strength * self.length
        return voltage_drop
    
    def voltage_drop_per_rung(self):
        if self.rungs == None:
            raise ValueError('Number of rungs undefined.')
        else:
            voltage_drop_per_rung = self.voltage_drop() / self.rungs
            return voltage_drop_per_rung
        
class voltage_setting:
    """This class defines the relationship between two channels, two regions, or an interface.
    Setting voltages using variables has been problematic due to scope and reassignment."""
    
    def __init__(self, description, voltage):
        self.description = description
        self.voltage = voltage
    
    def set_voltage(self, new_voltage):
        self.voltage = new_voltage

In [3]:
CIF = region('CIF', 10.87, 0)
CSSO = channel('CSSO', 0)
CIF_CL_to_CCSO_drop = voltage_setting('CIF CL to CSSO drop', 0)
CIF_CL = channel('CIFCL', 0)
CIF_to_CIF_CL_drop = voltage_setting('CIF to CIF CL drop', 0)
CIF_dc_out = channel('CIF_dc_out', 0)
CIF_dc_in = channel('CIF_dc_in', 0)

# Define the region and channels of SLIM 2
SLIM2 = region('SLIM2', 38.1, 0, rungs=250)
SLIM2_dc_in = channel('SLIM2_dc_in', 0)
SLIM2_dc_out = channel('SLIM2_dc_out', 0)
M8_sg = channel('Module 8 switch guard', 0)
SLIM2_to_CIF_drop = voltage_setting('SLIM2 to CIF drop', 0)
M8_sg_bias = voltage_setting('8M Switch Guard Bias', 0)

#Define channels in the interface region
M8_orth_out = channel('M8_orth_out', 0)
M8_orth_in = channel('M8_orth_in', 0)
M8_orth = region('M8_orth', 3.5052, 0)
SLIM2_8M_interface = 0
M8_orth_to_SLIM2 = voltage_setting('M8 Orth to SLIM2 drop', 0)

#Define channels in the interface region
M7_orth_out = channel('M7_orth_out', 0)
M7_orth_in = channel('M7_orth_in', 0)
M7_orth = region('M7_orth', 3.5052, 0)
SLIM1_7M_interface = 0

# #Define drops in the region
M7_to_M8_drop = voltage_setting('M7 to M8 drop', 0)

#Define channels and regions
SLIM1_dc_out = channel('SLIM1_dc_out', 0)
SLIM1_dc_in = channel('SLIM1_dc_in', 0)
M7_sg = channel('Module 7 switch guard', 0)
SLIM1 = region('SLIM1', 45.72, 0, rungs=300)

#Define interface and voltage drops
SLIM1_to_7M_orth_drop = voltage_setting('SLIM1 to 7M drop', 0)
M7_sg_bias = voltage_setting('7M Switch Guard Bias', 0)

EMPTY = channel('Empty Channel', 0)

In [4]:
Falkor_channels = {
    'channel_1' : CIF_dc_in,
    'channel_2' : CIF_dc_out,
    'channel_3' : CIF_CL
}
MIPS_2 = {
    'channel_1' : EMPTY,
    'channel_2' : EMPTY,
    'channel_3' : EMPTY,
    'channel_4' : EMPTY,
    'channel_5' : EMPTY,
    'channel_6' : M7_sg,
    'channel_7' : M7_orth_in,
    'channel_8' : M7_orth_out,
    'channel_9' : SLIM1_dc_out,
    'channel_10' : M8_sg,
    'channel_11' : M8_orth_in,
    'channel_12' : M8_orth_out,
    'channel_13' : SLIM2_dc_in,
    'channel_14' : SLIM2_dc_out,
    'channel_15' : EMPTY, 
    'channel_16' : EMPTY,
}
MIPS_1 = {
    'channel_1' : EMPTY,
    'channel_2' : EMPTY,
    'channel_3' : EMPTY,
    'channel_4' : EMPTY,
    'channel_5' : EMPTY,
    'channel_6' : EMPTY,
    'channel_7' : EMPTY,
    'channel_8' : EMPTY,
    'channel_9' : EMPTY,
    'channel_10' : EMPTY,
    'channel_11' : SLIM1_dc_in,
    'channel_12' : EMPTY,
    'channel_13' : EMPTY,
    'channel_14' : EMPTY,
    'channel_15' : EMPTY, 
    'channel_16' : EMPTY,
}

In [5]:
def set_voltages():
    #Set CIF voltages
    CIF_CL.set_voltage(CSSO.voltage + CIF_CL_to_CCSO_drop.voltage)
    CIF_dc_out.set_voltage(CIF_CL.voltage + CIF_to_CIF_CL_drop.voltage)
    CIF_dc_in.set_voltage(CIF_dc_out.voltage + (CIF.voltage_drop()))

    #Set SLIM2 voltages
    SLIM2_dc_out.set_voltage(CIF_dc_in.voltage + SLIM2_to_CIF_drop.voltage)
    SLIM2_dc_in.set_voltage(SLIM2_dc_out.voltage + (SLIM2.field_strength * SLIM2.length))
    global SLIM2_8M_interface
    SLIM2_8M_interface = (SLIM2_dc_in.voltage - (SLIM2.voltage_drop() * (25.5/250)))
    M8_sg.set_voltage(SLIM2_8M_interface + M8_sg_bias.voltage)

    #Set 8M voltages
    M8_orth_out.set_voltage(SLIM2_8M_interface + M8_orth_to_SLIM2.voltage)
    M8_orth_in.set_voltage(M8_orth_out.voltage + M8_orth.voltage_drop())

    #Set 7M voltages
    M7_orth_out.set_voltage(M8_orth_in.voltage + M7_to_M8_drop.voltage)
    M7_orth_in.set_voltage(M7_orth_out.voltage + M7_orth.voltage_drop())
    global SLIM1_7M_interface
    SLIM1_7M_interface = M7_orth_in.voltage + SLIM1_to_7M_orth_drop.voltage
    M7_sg.set_voltage(SLIM1_7M_interface + M8_sg_bias.voltage)

    #Set voltages
    SLIM1_dc_out.set_voltage(SLIM1_7M_interface - SLIM1.voltage_drop()*(25.5/300))
    SLIM1_dc_in.set_voltage(SLIM1_dc_out.voltage + SLIM1.voltage_drop())

In [40]:
LARGE_FONT = ("Verdana", 10)
style.use("ggplot")
    
def animate(i):
    CIF_loc = [0,-10.87]
    CIF_vol = [CIF_dc_out.voltage, CIF_dc_in.voltage]
    SLIM2_loc = [-10.87, -48.97]
    SLIM2_vol = [SLIM2_dc_out.voltage, SLIM2_dc_in.voltage]
    M8_orth_loc = [-45.0838,-48.589]
    M8_orth_vol = [M8_orth_out.voltage, M8_orth_in.voltage]
    M7_orth_loc = [-48.589, -52.0942]
    M7_orth_vol = [M7_orth_out.voltage, M7_orth_in.voltage]
    SLIM1_loc = [-48.208,-93.928]
    SLIM1_vol = [SLIM1_dc_out.voltage, SLIM1_dc_in.voltage]
    a0.clear()
    a1.clear()
    a0.plot(CIF_loc, CIF_vol,label='CIF')
    a0.plot(SLIM2_loc, SLIM2_vol, label='SLIM2')
    a0.plot(M8_orth_loc, M8_orth_vol, label='8M Orth')
    a0.plot(M7_orth_loc, M7_orth_vol, label='7M Orth')
    a0.plot(SLIM1_loc, SLIM1_vol, label='SLIM1')
    a0.legend()
    #a0.title('Drift Region Voltages')
    a1.plot(M8_orth_loc, M8_orth_vol, label='8M Orth')
    a1.plot(M7_orth_loc, M7_orth_vol, label='7M Orth')
    a1.scatter(-45.0838, SLIM2_8M_interface)
    a1.scatter(-52.0942, SLIM1_7M_interface)
    #a1.title('Orthogonal Region Voltages')
    
f = Figure(figsize=(10,5))
gs = gridspec.GridSpec(1,2, width_ratios=[3,2])
a0 = f.add_subplot(gs[0])
a1 = f.add_subplot(gs[1])

entry_objects = [CSSO, CIF_CL_to_CCSO_drop, CIF_to_CIF_CL_drop, CIF, SLIM2_to_CIF_drop, SLIM2, 
                 M8_sg_bias, M8_orth, M8_orth_to_SLIM2, M7_to_M8_drop, M7_orth, SLIM1_to_7M_orth_drop,
                 M7_sg_bias, SLIM1]


class SLIMVoltagesApplication(tk.Tk):
    
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        #tk.Tk.iconbitmap(self, default="insertthislater")
        tk.Tk.wm_title(self, "SLIM Voltages")
        
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {}
              
        for F in (VoltageControl,SignalTable):
        
            frame = F(container, self)

            self.frames[F] = frame
            
            frame.grid(row=0, column=0, sticky="nsew")
    
        self.show_frame(VoltageControl)
    
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
        

class VoltageControl(tk.Frame):
    
    def __init__(self, parent, controller): 
        tk.Frame.__init__(self, parent)
        button_dead = ttk.Button(self, text="Voltage Control")
        button_dead.grid(column=0, row=0)
        button = ttk.Button(self, text="Signal Tables", 
                            command=lambda: controller.show_frame(SignalTable))
        button.grid(column=1, row=0)
        
        #a frame to put the voltage setting entries
        voltage_entries_frame = tk.Frame(self)
        voltage_entries_frame.grid(column=0,row=1, padx=10)
                
        self.entries = []
        
        for n in range(len(entry_objects)):
            # create left side info labels
            tk.Label(voltage_entries_frame, text="%s: " % entry_objects[n].description).grid(row=n, column=0, pady=5, sticky="E")
            # create entries list
            self.entries.append(tk.Entry(voltage_entries_frame, width=10))
            # grid layout the entries
            self.entries[n].grid(row=n, column=1)
            # bind the entries return key pressed to an action
            self.entries[n].bind('<Tab>', partial(self.action, n))
        
        set_voltages_btn = tk.Button(voltage_entries_frame, text="Set Voltages", command=set_voltages)
        set_voltages_btn.grid(column=1, row=(len(entry_objects)+1))
        update_inputs_btn = tk.Button(voltage_entries_frame, text="Update Inputs", command=self.update_inputs)
        update_inputs_btn.grid(column=1, row=(len(entry_objects)+2))
    
        #a frame to display the voltage values of the MIPS inputs
        self.inputs_frame = tk.Frame(self)
        self.inputs_frame.grid(column=1,row=1,padx=10)
        
        #the canvas for the graph of field strengths and voltage settings
        canvas = FigureCanvasTkAgg(f, self)
        canvas.draw()
        canvas.get_tk_widget().grid(column=5, row=1)
        
        self.update_inputs()

    #methods
    def action(self, ix, event):
        '''this entry return key pressed'''
        try:
            new_voltage = float(self.entries[ix].get())
            if type(entry_objects[ix]) == region:
                entry_objects[ix].set_field_strength(new_voltage)
            else:
                entry_objects[ix].set_voltage(new_voltage)
        except ValueError:
            tk.messagebox.showerror("Oops!", "Value Error. Please enter a number.")
    
    def update_inputs(self):
        '''gives new voltages to the inputs'''
        incr = 1
        for n in MIPS_1:
            tk.Label(self.inputs_frame, text="MIPS1 %s:" % str(n),bg='red').grid(row=incr, column=0,sticky="EW")
            tk.Label(self.inputs_frame, text=str(np.around(MIPS_1[n].voltage))).grid(row=incr, column=1,sticky="EW")
            incr+=1
        for n in MIPS_2:
            tk.Label(self.inputs_frame, text="MIPS2 %s:" % str(n),bg='blue').grid(row=incr, column=0,sticky="EW")
            tk.Label(self.inputs_frame, text=str(np.around(MIPS_2[n].voltage))).grid(row=incr, column=1, sticky="EW")
            incr+=1
        for n in Falkor_channels:
            tk.Label(self.inputs_frame, text="MIPS3 %s:" % str(n),bg='green').grid(row=incr, column=0,sticky="EW")
            tk.Label(self.inputs_frame, text=str(np.around(Falkor_channels[n].voltage))).grid(row=incr, column=1,sticky="EW")
            incr+=1 
        

class SignalTable(tk.Frame):
    
    def __init__(self, parent, controller): 
        tk.Frame.__init__(self, parent)
        button = ttk.Button(self, text="Voltage Control", 
                            command=lambda: controller.show_frame(VoltageControl))
        button.grid(row=0, column=0)
        button_dead = ttk.Button(self, text="Signal Tables")
        button_dead.grid(row=0,column=1, sticky="nsew")


app = SLIMVoltagesApplication()
print("Application initialized")
ani = animation.FuncAnimation(f, animate, interval=1000)
print("Animation started")
app.mainloop()
print("Mainloop exited")

Application initialized
Animation started
Mainloop exited


In [ ]:
for i in Falkor_channels:
    print(Falkor_channels[i].description)

In [ ]:
for i in Falkor_channels:
    print(Falkor_channels[i].voltage)

In [ ]:
for i in MIPS_2:
    print(MIPS_2[i].voltage)

In [ ]:
for i in MIPS_1:
    print(MIPS_1[i].voltage)